# Household Location Choice Model (HLCM) for Single and Multi Family Housing  


Arezoo Besharati, Paul Waddell, UrbanSim, July 2018 

This notebook demonstrates the use of the LargeMultinomialLogit model template to construct, estimate, and evaluate a Household Location Choice Model for the San Francisco Bay Area.

In the process of developing the model, we also demonstrate some data checking and transformations to improve the model.

The model structure and specification are informed and limited by the available data, which is based on the data used by the Metropolitan Transportation Commission for their operational model.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preliminaries" data-toc-modified-id="Preliminaries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preliminaries</a></span><ul class="toc-item"><li><span><a href="#Load-data" data-toc-modified-id="Load-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load data</a></span></li><li><span><a href="#Initialize-Networks" data-toc-modified-id="Initialize-Networks-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Initialize Networks</a></span></li><li><span><a href="#Explore-the-data" data-toc-modified-id="Explore-the-data-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Explore the data</a></span></li><li><span><a href="#Data-Cleaning" data-toc-modified-id="Data-Cleaning-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Data Cleaning</a></span></li><li><span><a href="#Perform-desired-variable-creations-and-transformations¶" data-toc-modified-id="Perform-desired-variable-creations-and-transformations¶-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Perform desired variable creations and transformations¶</a></span></li><li><span><a href="#Check-Distributions" data-toc-modified-id="Check-Distributions-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Check Distributions</a></span></li><li><span><a href="#Check-for-Multicollinearity" data-toc-modified-id="Check-for-Multicollinearity-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Check for Multicollinearity</a></span></li></ul></li><li><span><a href="#Model-Estimation" data-toc-modified-id="Model-Estimation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model Estimation</a></span><ul class="toc-item"><li><span><a href="#HLCM-for-Multi-Family" data-toc-modified-id="HLCM-for-Multi-Family-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>HLCM for Multi-Family</a></span></li><li><span><a href="#HLCM-for-Single-Family" data-toc-modified-id="HLCM-for-Single-Family-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>HLCM for Single-Family</a></span></li></ul></li></ul></div>

## Preliminaries

In [1]:
import os; os.chdir('../../')
import numpy as np, pandas as pd 
import matplotlib.pyplot as plt
import warnings;
warnings.simplefilter('ignore')
%load_ext autoreload
%autoreload 2

In [2]:
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
import orca
import seaborn as sns
%matplotlib notebook

In [3]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Load data

In [4]:
# Load any script-based Orca registrations
from scripts import datasources
from scripts import models

In [5]:
orca.list_tables()
#orca.list_steps()

['parcels',
 'buildings',
 'craigslist',
 'rentals',
 'nodessmall',
 'nodeswalk',
 'units',
 'households',
 'persons',
 'jobs']

In [40]:
orca.broadcast('nodeswalk', 'rentals', cast_index=True, onto_on='node_id_walk')
orca.broadcast('nodeswalk', 'parcels', cast_index=True, onto_on='node_id_walk')
orca.broadcast('nodessmall', 'rentals', cast_index=True, onto_on='node_id_small')
orca.broadcast('nodessmall', 'parcels', cast_index=True, onto_on='node_id_small')
orca.list_broadcasts()

[('parcels', 'buildings'),
 ('buildings', 'units'),
 ('units', 'households'),
 ('households', 'persons'),
 ('nodeswalk', 'rentals'),
 ('nodeswalk', 'parcels'),
 ('nodessmall', 'rentals'),
 ('nodessmall', 'parcels')]

In [ ]:
#mm.list_steps()

In [ ]:
#mm.initialize()

### Initialize Networks

Note that we have separately created a set of network accessibility variables and stored them. They are registered a orca objects.

In [ ]:
#%%capture
#orca.run(["initialize_network_walk"])

In [ ]:
#%%capture
#orca.run(["initialize_network_small"])

### Explore the data

In [37]:
nodeswalk = orca.get_table('nodeswalk').to_frame()
nodessmall = orca.get_table('nodessmall').to_frame()

In [38]:
nodeswalk.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
units_500_walk,619889.00,238.93,377.17,0.00,5.00,91.00,318.00,2317.12
sqft_unit_500_walk,619889.00,1221.43,864.01,0.00,449.75,1327.47,1760.37,3698.59
singles_500_walk,619889.00,59.38,125.91,0.00,0.00,13.00,61.00,868.00
elderly_hh_500_walk,619889.00,39.19,61.74,0.00,0.00,15.00,52.00,363.00
children_500_walk,619889.00,110.65,151.64,0.00,1.00,46.00,164.00,755.00
units_sf_500_walk,619889.00,116.08,149.80,0.00,2.00,52.00,179.00,719.00
units_mf_500_walk,619889.00,107.51,257.35,0.00,0.00,0.00,86.00,1654.00
pop_500_walk,619889.00,543.32,783.63,0.00,11.00,224.00,770.00,4352.12
hh_500_walk,619889.00,212.32,333.00,0.00,4.00,81.00,284.00,2045.00
poor_500_walk,619889.00,55.35,105.98,0.00,0.00,13.00,59.00,643.00


In [39]:
nodessmall.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
units_10000,30826.00,111521.06,86779.17,0.00,43964.50,96821.50,163121.75,377070.25
units_sf_10000,30826.00,60712.51,37339.24,0.00,31221.50,59398.00,90068.50,136949.50
units_mf_10000,30826.00,45940.89,45879.03,0.00,10558.00,32530.50,69672.00,196744.00
pop_10000,30826.00,259908.40,187431.24,0.00,106498.75,231983.00,392361.00,763247.25
hh_10000,30826.00,98968.73,77650.94,0.00,39192.00,85626.00,143069.50,343047.25
poor_10000,30826.00,24228.51,22002.93,0.00,7992.25,17596.50,34093.75,94001.50
renters_10000,30826.00,47192.46,47728.75,0.00,13226.75,33240.50,65630.00,214109.25
avg_income_10000,30826.00,111123.63,30051.59,0.00,93095.60,111716.90,129615.74,174412.79
jobs_10000,30826.00,118174.96,107815.26,0.00,28940.25,92305.00,176939.00,412326.25
avg_rent_10000,30826.00,2630.09,753.54,0.00,2328.39,2704.73,2981.97,4020.42


In [7]:
parcel = orca.get_table('parcels').to_frame()
parcel.columns

Index(['development_type_id', 'land_value', 'acres', 'county_id', 'zone_id',
       'proportion_undevelopable', 'tax_exempt_status', 'apn',
       'parcel_id_local', 'geom_id', 'imputation_flag', 'x', 'y', 'shape_area',
       'block_id', 'node_id', 'node_id_small', 'node_id_walk'],
      dtype='object')

In [8]:
bld = orca.get_table('buildings').to_frame()
bld.columns

Index(['parcel_id', 'development_type_id', 'improvement_value',
       'residential_units', 'residential_sqft', 'sqft_per_unit',
       'non_residential_sqft', 'building_sqft', 'nonres_rent_per_sqft',
       'res_price_per_sqft', 'stories', 'year_built', 'redfin_sale_price',
       'redfin_sale_year', 'redfin_home_type', 'costar_property_type',
       'costar_rent', 'building_type_id', 'res_sqft_per_unit'],
      dtype='object')

In [9]:
hh = orca.get_table('households').to_frame()
hh.columns.tolist()

In [46]:
hh['hh_random'] = np.random.uniform(0,1,len(hh))

In [11]:
hh.head()

,serialno,persons,building_type,cars,income,race_of_head,hispanic_head,age_of_head,workers,state,...,tract,block_group,children,tenure,recent_mover,block_group_id,single_family,unit_id,building_id,hh_random
household_id,,,,,,,,,,,,,,,,,,,,,
0,2010000487191,1,6.00,1.00,85000.00,1,no,47,1.00,6,...,500901,1,0.00,2,0,60855009011,False,1711366,409174,0.81
1,2013000554587,1,9.00,1.00,27000.00,6,no,52,1.00,6,...,500901,1,0.00,2,0,60855009011,False,1711818,1579443,0.38
2,2011001140920,2,2.00,1.00,6000.00,6,no,60,1.00,6,...,500901,1,0.00,2,0,60855009011,True,1711727,1365639,0.81
3,2012001376432,1,3.00,1.00,28000.00,8,yes,51,1.00,6,...,500901,1,0.00,2,0,60855009011,True,1711548,647211,0.19
4,2013000186929,1,6.00,1.00,10000.00,1,no,64,0.00,6,...,500901,1,0.00,2,0,60855009011,False,1711121,309586,0.69


In [19]:
hh.describe()

,serialno,persons,building_type,cars,income,race_of_head,age_of_head,workers,children,tenure,recent_mover,unit_id,building_id,hh_random
count,2677468.00,2677468.00,2677468.00,2677468.00,2677468.00,2677468.00,2677468.00,2677468.00,2677468.00,2677468.00,2677468.00,2677468.00,2677468.00,2677468.00
mean,2011048463205.79,2.64,3.59,1.88,113128.36,2.84,52.97,1.30,0.56,1.43,0.04,1288427.13,858095.84,0.50
std,1416905562.31,1.57,2.34,1.14,107952.60,2.66,15.69,1.00,0.97,0.49,0.19,849779.84,561165.80,0.29
min,2009000000022.00,1.00,1.00,0.00,-11600.00,1.00,15.00,0.00,0.00,1.00,0.00,-1.00,-1.00,0.00
25%,2010000424551.00,1.00,2.00,1.00,40320.00,1.00,41.00,1.00,0.00,1.00,0.00,540970.75,359954.75,0.25
50%,2011000848265.00,2.00,2.00,2.00,84000.00,1.00,53.00,1.00,0.00,1.00,0.00,1272990.50,837858.50,0.50
75%,2012001211486.00,4.00,5.00,2.00,150100.00,6.00,63.00,2.00,1.00,2.00,0.00,2024749.25,1341263.25,0.75
max,2013001492566.00,20.00,10.00,6.00,1397000.00,9.00,94.00,13.00,9.00,2.00,1.00,2781489.00,1843272.00,1.00


In [27]:
hh.dtypes

serialno            int64
persons             int64
building_type     float64
cars              float64
income            float64
race_of_head        int64
hispanic_head      object
age_of_head         int64
workers           float64
state              object
county             object
tract              object
block_group        object
children          float64
tenure              int64
recent_mover        int64
block_group_id     object
single_family        bool
unit_id             int64
building_id         int64
hh_random         float64
dtype: object

In [12]:
hh_f = hh[(hh['building_type'] > 2) & (hh['hh_random'] < .2) & (hh['recent_mover'] == 1) \
         & (hh['income'] > 0) & (hh['income'] < 500000)]
len(hh_f)

15468

In [34]:
parcel.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
development_type_id,1956207.00,2.61,5.31,-1.00,1.00,1.00,1.00,25.00
land_value,1956207.00,234801.87,1610709.58,0.00,39957.00,100800.00,225434.50,1393463526.00
acres,1956207.00,2.29,36.12,0.00,0.11,0.15,0.23,23351.99
county_id,1956207.00,53.94,38.09,1.00,13.00,75.00,85.00,97.00
zone_id,1956207.00,829.32,412.98,1.00,492.00,844.00,1189.00,1454.00
proportion_undevelopable,1956207.00,0.00,0.04,0.00,0.00,0.00,0.00,1.00
tax_exempt_status,1956207.00,0.01,0.11,0.00,0.00,0.00,0.00,1.00
geom_id,1956207.00,8795113526279.49,5076215226459.58,17066885.00,4405537788456.00,8793872715803.00,13187559413395.50,17592170803819.00
x,1956207.00,-122.18,0.30,-123.53,-122.41,-122.14,-121.95,-121.21
y,1956207.00,37.77,0.36,36.90,37.48,37.75,37.99,38.86


In [36]:
bld.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
parcel_id,1824229.00,1012860.54,593806.46,26.00,495850.00,1002691.00,1532117.00,2054501.00
development_type_id,1824229.00,1.84,3.44,-1.00,1.00,1.00,1.00,24.00
improvement_value,1824229.00,318687.80,3571670.95,0.00,82675.03,167553.52,285342.35,3355514162.00
residential_units,1824229.00,1.53,6.14,0.00,1.00,1.00,1.00,1912.00
residential_sqft,1824229.00,2657.11,30124.08,0.00,1230.00,1754.00,2400.00,14580000.00
sqft_per_unit,1824229.00,1669.29,1063.48,0.00,1176.00,1564.00,2029.42,30000.00
non_residential_sqft,1824229.00,1280.86,23922.26,0.00,0.00,0.00,0.00,16573099.00
building_sqft,1824229.00,3784.46,34945.68,0.00,1320.00,1858.00,2524.00,14580000.00
nonres_rent_per_sqft,1824229.00,0.92,4.05,0.00,0.00,0.00,0.00,128.96
res_price_per_sqft,1824229.00,292.72,264.77,0.00,151.87,254.46,369.91,50161.09


In [41]:
df = orca.merge_tables(target = 'buildings', tables = ['buildings','parcels','nodeswalk','nodessmall'])

In [42]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
parcel_id,1824229.00,1012860.54,593806.46,26.00,495850.00,1002691.00,1532117.00,2054501.00
development_type_id,1824229.00,1.84,3.44,-1.00,1.00,1.00,1.00,24.00
improvement_value,1824229.00,318687.80,3571670.95,0.00,82675.03,167553.52,285342.35,3355514162.00
residential_units,1824229.00,1.53,6.14,0.00,1.00,1.00,1.00,1912.00
residential_sqft,1824229.00,2657.11,30124.08,0.00,1230.00,1754.00,2400.00,14580000.00
sqft_per_unit,1824229.00,1669.29,1063.48,0.00,1176.00,1564.00,2029.42,30000.00
non_residential_sqft,1824229.00,1280.86,23922.26,0.00,0.00,0.00,0.00,16573099.00
building_sqft,1824229.00,3784.46,34945.68,0.00,1320.00,1858.00,2524.00,14580000.00
nonres_rent_per_sqft,1824229.00,0.92,4.05,0.00,0.00,0.00,0.00,128.96
res_price_per_sqft,1824229.00,292.72,264.77,0.00,151.87,254.46,369.91,50161.09


In [32]:
hh_f.dtypes

serialno            int64
persons             int64
building_type     float64
cars              float64
income            float64
race_of_head        int64
hispanic_head      object
age_of_head         int64
workers           float64
state              object
county             object
tract              object
block_group        object
children          float64
tenure              int64
recent_mover        int64
block_group_id     object
single_family        bool
unit_id             int64
building_id         int64
hh_random         float64
dtype: object

In [43]:
hh_f_data = hh_f.merge(df, left_on='building_id', right_index=True)

In [44]:
hh_f_data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
serialno,13135.00,2011089645440.15,1442111691.19,2009000001284.00,2010000420917.00,2011000997364.00,2012001322354.00,2013001492332.00
persons,13135.00,2.18,1.20,1.00,1.00,2.00,3.00,14.00
building_type,13135.00,6.46,2.06,3.00,5.00,7.00,8.00,10.00
cars,13135.00,1.34,0.81,0.00,1.00,1.00,2.00,6.00
income,13135.00,95923.52,77543.61,4.00,40000.00,78000.00,130000.00,498000.00
race_of_head,13135.00,3.41,2.81,1.00,1.00,1.00,6.00,9.00
age_of_head,13135.00,34.26,12.17,16.00,27.00,31.00,36.00,94.00
workers,13135.00,1.40,0.78,0.00,1.00,1.00,2.00,7.00
children,13135.00,0.39,0.78,0.00,0.00,0.00,0.00,6.00
tenure,13135.00,1.90,0.30,1.00,2.00,2.00,2.00,2.00


In [39]:
hh_f.head()

,serialno,persons,building_type,cars,income,race_of_head,hispanic_head,age_of_head,workers,state,...,tract,block_group,children,tenure,recent_mover,block_group_id,single_family,unit_id,building_id,hh_random
household_id,,,,,,,,,,,,,,,,,,,,,
169,2009001308675,2,7.00,1.00,1200.00,6,no,21,1.00,6,...,500901,1,0.00,2,1,60855009011,False,1711656,1364704,0.09
258,2012000897474,3,4.00,2.00,27000.00,9,no,23,2.00,6,...,500901,1,0.00,2,1,60855009011,False,1711344,409172,0.15
264,2010000652157,4,8.00,1.00,12120.00,6,no,22,0.00,6,...,500901,1,0.00,2,1,60855009011,False,1711177,309592,0.10
304,2013000035552,1,6.00,1.00,50000.00,1,no,32,1.00,6,...,500901,1,0.00,2,1,60855009011,False,1711143,309587,0.12
333,2013001130635,1,7.00,2.00,8000.00,9,no,23,1.00,6,...,500901,1,0.00,2,1,60855009011,False,1711026,231282,0.02


In [ ]:
#bld.groupby('building_type_id')['residential_units'].sum()

In [ ]:
#bld.describe().transpose()

In [ ]:
#bld.quantile(.99)

In [ ]:
#bld[['residential_units','residential_sqft', 'non_residential_sqft']].sum()

In [ ]:
#bld.building_type_id.value_counts()

In [ ]:
# hh_sf=hh[hh['building_type']==2]
# hh_mf=hh[hh['building_type']!=2]

### Data Cleaning

In [ ]:
# nodeswalk = orca.get_table('nodeswalk').to_frame()
# nodessmall = orca.get_table('nodessmall').to_frame()
#print(nodeswalk.columns.tolist())
#print(nodessmall.columns.tolist())

In [ ]:
#nodessmall.describe().transpose()

In [ ]:
# nodessmall_upper = nodessmall.quantile(.99)
#nodessmall_upper

In [ ]:
# nodessmall_clipped = nodessmall.clip_upper(nodessmall_upper, axis=1) 
# nodessmall_clipped.describe().transpose()
# orca.add_table('nodessmall', nodessmall_clipped);

In [ ]:
# nodeswalk.describe().transpose()
# nodeswalk_upper = nodeswalk.quantile(.99)
#nodeswalk_upper

In [ ]:
# nodeswalk_clipped = nodeswalk.clip_upper(nodeswalk_upper, axis=1) 
# nodeswalk_clipped.describe().transpose()
# orca.add_table('nodeswalk', nodeswalk_clipped);

### Perform desired variable creations and transformations¶

In [47]:
# scale income and create race dummies
hh.income_k = hh.income/1000
hh.white = (hh.race_of_head == 1).astype(int)
hh.black = (hh.race_of_head == 2).astype(int)
hh.asian = (hh.race_of_head == 6).astype(int)
hh.hisp = (hh.hispanic_head == 'yes').astype(int)
hh.single = (hh.persons == 1).astype(int)
hh.elderly = (hh.age_of_head > 65).astype(int)

  
# building_type dummies
bld.single_family = (bld.building_type_id == 1).astype(int)
bld.multi_family = (bld.building_type_id == 3).astype(int)
bld.mixed_use = (bld.building_type_id > 3).astype(int)

# add the columns

orca.add_column('households', 'income_k', hh.income_k)
orca.add_column('households', 'white', hh.white)
orca.add_column('households', 'black', hh.black)
orca.add_column('households', 'asian', hh.asian)
orca.add_column('households', 'hispanic', hh.hisp)
orca.add_column('households', 'elderly', hh.elderly)
orca.add_column('households', 'single', hh.single)
orca.add_column('households', 'hh_random', hh.hh_random)

orca.add_column('buildings', 'single_family', bld.single_family)
orca.add_column('buildings', 'multi_family', bld.multi_family)
orca.add_column('buildings', 'mixed_use', bld.mixed_use);

In [ ]:
# nodeswalk['prop_children_500_walk'] = (nodeswalk['children_500_walk'] > 0).astype(int) / nodeswalk['hh_500_walk']
# nodeswalk['prop_singles_500_walk'] = nodeswalk['singles_500_walk'] / nodeswalk['hh_500_walk']
# nodeswalk['prop_elderly_500_walk'] = nodeswalk['elderly_hh_500_walk'] / nodeswalk['hh_500_walk']
# nodeswalk['prop_black_500_walk'] = nodeswalk['pop_black_500_walk'] / nodeswalk['pop_500_walk']
# nodeswalk['prop_white_500_walk'] = nodeswalk['pop_white_500_walk'] / nodeswalk['pop_500_walk']
# nodeswalk['prop_asian_500_walk'] = nodeswalk['pop_asian_500_walk'] / nodeswalk['pop_500_walk']
# nodeswalk['prop_hisp_500_walk'] = nodeswalk['pop_hisp_500_walk'] / nodeswalk['pop_500_walk']
# nodeswalk['prop_rich_500_walk'] = nodeswalk['rich_500_walk'] / nodeswalk['pop_500_walk']
# nodeswalk['prop_poor_500_walk'] = nodeswalk['poor_500_walk'] / nodeswalk['pop_500_walk']

# nodeswalk['prop_children_1500_walk'] = (nodeswalk['children_1500_walk'] > 0).astype(int) / nodeswalk['hh_1500_walk']
# nodeswalk['prop_singles_1500_walk'] = nodeswalk['singles_1500_walk'] / nodeswalk['hh_1500_walk']
# nodeswalk['prop_elderly_1500_walk'] = nodeswalk['elderly_hh_1500_walk'] / nodeswalk['hh_1500_walk']
# nodeswalk['prop_black_1500_walk'] = nodeswalk['pop_black_1500_walk'] / nodeswalk['pop_1500_walk']
# nodeswalk['prop_white_1500_walk'] = nodeswalk['pop_white_1500_walk'] / nodeswalk['pop_1500_walk']
# nodeswalk['prop_asian_1500_walk'] = nodeswalk['pop_asian_1500_walk'] / nodeswalk['pop_1500_walk']
# nodeswalk['prop_hisp_1500_walk'] = nodeswalk['pop_hisp_1500_walk'] / nodeswalk['pop_1500_walk']
# nodeswalk['prop_rich_1500_walk'] = nodeswalk['rich_1500_walk'] / nodeswalk['pop_1500_walk']
# nodeswalk['prop_poor_1500_walk'] = nodeswalk['poor_1500_walk'] / nodeswalk['pop_1500_walk']


# orca.add_column('nodeswalk', 'prop_children_500_walk', nodeswalk['prop_children_500_walk'])
# orca.add_column('nodeswalk', 'prop_singles_500_walk', nodeswalk['prop_singles_500_walk'])
# orca.add_column('nodeswalk', 'prop_elderly_500_walk', nodeswalk['prop_elderly_500_walk'])
# orca.add_column('nodeswalk', 'prop_black_500_walk', nodeswalk['prop_black_500_walk'])
# orca.add_column('nodeswalk', 'prop_white_500_walk', nodeswalk['prop_white_500_walk'])
# orca.add_column('nodeswalk', 'prop_asian_500_walk', nodeswalk['prop_asian_500_walk'])
# orca.add_column('nodeswalk', 'prop_hisp_500_walk', nodeswalk['prop_hisp_500_walk'])
# orca.add_column('nodeswalk', 'prop_rich_500_walk', nodeswalk['prop_rich_500_walk'])
# orca.add_column('nodeswalk', 'prop_poor_500_walk', nodeswalk['prop_poor_500_walk'])

# orca.add_column('nodeswalk', 'prop_children_1500_walk', nodeswalk['prop_children_1500_walk'])
# orca.add_column('nodeswalk', 'prop_singles_1500_walk', nodeswalk['prop_singles_1500_walk'])
# orca.add_column('nodeswalk', 'prop_elderly_1500_walk', nodeswalk['prop_elderly_1500_walk'])
# orca.add_column('nodeswalk', 'prop_black_1500_walk', nodeswalk['prop_black_1500_walk'])
# orca.add_column('nodeswalk', 'prop_white_1500_walk', nodeswalk['prop_white_1500_walk'])
# orca.add_column('nodeswalk', 'prop_asian_1500_walk', nodeswalk['prop_asian_1500_walk'])
# orca.add_column('nodeswalk', 'prop_hisp_1500_walk', nodeswalk['prop_hisp_1500_walk'])
# orca.add_column('nodeswalk', 'prop_rich_1500_walk', nodeswalk['prop_rich_1500_walk'])
# orca.add_column('nodeswalk', 'prop_poor_1500_walk', nodeswalk['prop_poor_1500_walk'])

### Check Distributions

In [ ]:
# hh_income_upper = hh_singlefamily.income.quantile(0.99)
# len(hh[hh['income']>hh_income_upper])

In [ ]:
# %matplotlib notebook
# hh_sf.income.plot(kind='hist',bins=100)
# print(hh_sf.income.quantile([0.10,0.5,0.9,0.99,1]))

In [ ]:
# %matplotlib notebook
# hh_mf.income.plot(kind='hist',bins=100)
# print(hh_mf.income.quantile([0.10,0.5,0.9,0.99,1]))

In [ ]:
# %matplotlib notebook
# ax = sns.boxplot(x = hh_sf.income,palette="Set2", width=0.5)

In [ ]:
# %matplotlib notebook
# ax = sns.boxplot(x = hh_mf.income,palette="Set2", width=0.5)

In [ ]:
df = orca.merge_tables(target = 'buildings', tables = ['buildings','parcels','nodeswalk','nodessmall'])
#df.columns.tolist()
df_sf= df[df['residential_units']==1]
df_mf= df[df['residential_units']>1]

In [ ]:
# %matplotlib notebook
# df_sf.avg_income_1500_walk.plot(kind='hist',bins=100)
# plt.show()
# print(df_sf.avg_income_1500_walk.quantile([0.10,0.5,0.9,0.99,1]))

In [ ]:
# %matplotlib notebook
# df_mf.avg_income_1500_walk.plot(kind='hist',bins=100)
# plt.show()
# print(df_mf.avg_income_1500_walk.quantile([0.10,0.5,0.9,0.99,1]))

In [ ]:
#%matplotlib notebook
#(df_mf.prop_black_500_walk).plot(kind='hist',bins=100)
#(df_mf.prop_asian_500_walk).plot(kind='hist',bins=100)
#(df_mf.prop_white_500_walk).plot(kind='hist',bins=100)
#(df_mf.prop_hisp_500_walk).plot(kind='hist',bins=100)


### Check for Multicollinearity

In [ ]:
# %matplotlib inline
# plt.subplots(figsize=(20, 20))


# # Create correlation matrix
# corr_matrix = df_sf.corr().abs()

# # Select upper triangle of correlation matrix
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# high_corr = [column for column in upper.columns if any(upper[column] > 0.98)]
# X= df_sf[high_corr]
# sns.heatmap(X.corr(), annot=True, cmap="RdYlGn")

In [ ]:
# %matplotlib inline
# plt.subplots(figsize=(20, 20))

# corr_matrix = df_mf.corr()

# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# high_corr = [column for column in upper.columns if any(upper[column] > 0.95)]
# X= df_mf[high_corr]
# plt.subplots(figsize=(20, 20))
# sns.heatmap(X.corr(), annot=True, cmap="RdYlGn")

In [ ]:
%matplotlib inline

selected_features = ['residential_units','res_price_per_sqft', 'pop_1500_walk', 'jobs_1500_walk',
                     'prop_singles_1500_walk','prop_elderly_1500_walk','prop_children_1500_walk',
                     'avg_income_1500_walk','prop_black_1500_walk', 'sqft_per_unit','prop_poor_1500_walk',
                     'jobs_25000','prop_white_1500_walk','prop_black_1500_walk','prop_asian_1500_walk',
                     'prop_hisp_1500_walk']

X = df_mf[selected_features]

plt.subplots(figsize=(12, 12))
sns.heatmap(X.corr(), annot=True, cmap="RdYlGn")
plt.show()

## Model Estimation

### HLCM for Multi-Family 

In [48]:
%%time
m = LargeMultinomialLogitStep()
m.choosers = ['households']
m.alternatives = ['buildings','parcels','nodeswalk','nodessmall']
m.choice_column = 'building_id'
m.alt_sample_size = 50

#Filters on choosers
m.chooser_filters = ['building_type > 2 &\
                      hh_random < .2 &\
                      recent_mover == 1 &\
                      0 <income < 500000']

#Filters on alternatives
m.alt_filters = ['residential_units > 1',
                 '0 < avg_income_500_walk < 500000',
                 'pop_1500_walk > 0',
                 'sqft_per_unit > 0']


m.model_expression = ' np.log(residential_units) + \
np.log1p(res_price_per_sqft) + \
np.log1p(sqft_per_unit) + \
np.log1p(income):np.log1p(sqft_per_unit) + \
np.log1p(jobs_1500_walk) + \
np.log1p(jobs_25000) + \
np.log(income):np.log(avg_income_1500_walk) + \
np.log1p(pop_1500_walk) + \
white:prop_white_500_walk + \
black:prop_black_500_walk + \
asian:prop_asian_500_walk + \
hispanic:prop_hisp_500_walk\
- 1'

m.name = 'hlcm'
m.tags = ['multi_family','test']
m.fit()

                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:         10,416
Model:         Multinomial Logit   Df Residuals:             10,404
Method:       Maximum Likelihood   Df Model:                     12
Date:                 2018-07-20   Pseudo R-squ.:             0.404
Time:                      18:08   Pseudo R-bar-squ.:         0.404
AIC:                  48,575.150   Log-Likelihood:      -24,275.575
BIC:                  48,662.163   LL-Null:             -40,747.632
                                                 coef   std err         z     P>|z|   Conf. Int.
------------------------------------------------------------------------------------------------
np.log(residential_units)                      1.2205     0.008   154.385     0.000             
np.log1p(res_price_per_sqft)                  -0.1663     0.011   -14.697     0.000             
np.log1p(sqft_per_unit)                       -0.2008     0.034    -

In [40]:
len(m._get_df(tables=m.choosers, filters=m.chooser_filters))

15308

### HLCM for Single-Family 

In [ ]:
%%time
m = LargeMultinomialLogitStep()
m.choosers = ['households']
m.alternatives = ['buildings','parcels','nodeswalk','nodessmall']
m.choice_column = 'building_id'
m.alt_sample_size = 50

#Filters on choosers
m.chooser_filters = ['building_type == 2 & household_id <3000000 & recent_mover == 1 & 0 <income < 1000000']

#Filters on alternatives
m.alt_filters = ['residential_units == 1',
                 '0 < avg_income_500_walk < 500000',
                 'sqft_per_unit > 0']

# np.log(residential_units) +
# np.log(income):np.log(avg_income_500_walk) + \
# np.log1p(income):np.log1p(rich_1500_walk) + \
# np.log1p(income):np.log1p(poor_1500_walk) + \
# np.log1p(persons):np.log1p(sqft_per_unit) + \

m.model_expression = ' \
np.log1p(res_price_per_sqft) + \
np.log1p(income):np.log1p(res_price_per_sqft) + \
np.log1p(sqft_per_unit) + \
np.log1p(income):np.log1p(sqft_per_unit) + \
np.log1p(acres) + \
np.log1p(income):np.log1p(acres) + \
np.log1p(pop_1500_walk) + \
np.log1p(jobs_1500_walk) + \
np.log1p(jobs_25000) + \
single:np.log1p(singles_1500_walk) + \
elderly:np.log1p(elderly_hh_1500_walk) + \
children:np.log1p(children_1500_walk) + \
white:np.log1p(pop_white_1500_walk) + \
black:np.log1p(pop_black_1500_walk) + \
asian:np.log1p(pop_asian_1500_walk) + \
hispanic:np.log1p(pop_hisp_1500_walk) \
- 1'

m.name = 'hlcm'
m.tags = ['single_family', 'test']
m.fit()